In [1]:
# main_nse.py
"""Main program for NSE
Date: 23-July-2019
Ver: 1.0
"""

##### Main Program for NSE
from z_helper import *
util.startLoop()

from chains_nse import *
from ohlcs import *
from sized_nse import *
from target_nse import *
from workout_nse import *
from x_4Capstocks import *

# from json
a = assign_var('nse') + assign_var('common')
for v in a:
    exec(v)

def ask_user():
    '''Asks the user for what needs to be done
    Arg: None
    Returns: 0 to 7 int'''
    # Get user input
    askmsg = "\nChoose from the following numbers:\n" + \
            "0) Prepare for morning trades\n" + \
            "1) Chain Generation\n" + \
            "2) OHLCs Generation\n" + \
            "3) Size the options\n" + \
            "4) Target preparation\n"+ \
            "5) Place morning trades\n" + \
            "6) Workout closing trades (DYNAMIC)\n" + \
            "7) Zip Capstocks BUYs\n\n" + \
            "...Or close window to abort\n\n"

    while True:
        try:
            ip = int(input(askmsg+'\n'))
        except ValueError:
            print("\nSorry, I didn't understand what you entered. Try again!\n")
            continue # Loop again
        if not ip in [0, 1, 2, 3, 4, 5, 6, 7]:
            print(f"\n{ip} is a wrong number! Choose any number from 0 to 7 and press <Enter>\n")
        else:
            break # success and exit loop
    
    return ip

# delete data and log files
def delete_all_data():
    '''Deletes all data and log files
    Arg: None Return: None'''
    folderpaths = ["../data/nse/", "../data/log/"]

    for folder in folderpaths:
        for files in listdir(folder):
            file_path = path.join(folder, files)
            try:
                if path.isfile(file_path):
                    unlink(file_path)
            except Exception as e:
                print(e)
                
    return None

# generate ohlcs
def make_ohlcs(ib, df_chains):
    '''Makes OHLCs for the underlying symbols
    Args:
        (ib) as connection object
        (df_chains) as DataFrame chain
    Returns: df_ohlcs as DataFrame of ohlcs and pickles them'''
    id_sym = df_chains.set_index('undId').symbol.to_dict()

    df_ohlcs = ohlcs(ib, id_sym, fspath, logpath)

    df_ohlcs.to_pickle(fspath+'ohlcs.pkl')
    
    return df_ohlcs

# get CAPSTOCK trades
def get_capstocks(cap_blacklist):
    '''Prepares list of Capstock Trades
    Arg: (cap_blacklist) as list of symbols with positions that need to be excluded
    Returns: DataFrame of capstock trades. Also makes a spreadsheet and watchlist'''
    
    df_cap=capstocks(cap_blacklist)
    
    return df_cap

# do all the functions
def do_all(ib):
    '''Does all the functions outlined
    Args: None
    Returns: None'''
    
    # delete all data and log files
    delete_all_data()
    print("Deleted all data and log files\n")
    
    # do all the functions
    df_chains=get_chains(nseweb=False)
    print("Got the chains\n")
    
    df_ohlcs=make_ohlcs(ib, df_chains)
    print("Made the OHLCs\n")
    
    df_sized=sized_nse(ib, df_chains, df_ohlcs)
    print("Sized the options\n")
    
    # Error in margins
    if len(df_sized[df_sized.margin.isnull()]) == len(df_sized):
        print("\nERROR: Margins unavailable. Please run sizing again!\n")
        return None
    
    df_targets = target_nse(ib, df_sized, blacklist, nse_assignment_limit)
    print("Build the targets\n")
    
    df_sell = df_targets
    df_buy = workout_nse(ib)

    return df_buy, df_sell

In [2]:
# userip.py
# the selecting user inputs
if __name__=='__main__':
    userip = ask_user()
    
    with get_connected('nse', 'live') as ib:
        if userip == 0: # Run all
            start = time.time()
            print("\nRunning ALL\n")
            df_buy, df_sell = do_all(ib)
            print(f"\nTook {codetime(time.time()-start)} to complete do_all\n")
            
        elif userip == 1: # Chain Generation
            start = time.time()
            print("\nGetting Chains\n")
            df_chains=get_chains(nseweb=False)
            print(f"\nGot option chains in {codetime(time.time()-start)}\n")
            
        elif userip == 2: # OHLC Generation
            start = time.time()
            print("\nGenerating OHLCs\n")
            df_chains = pd.read_pickle(fspath+'chains.pkl')
            df_ohlcs=make_ohlcs(ib, df_chains)
            print(f"\nOHLCs generated in {codetime(time.time()-start)}\n")
            
        elif userip == 3: # Size the options
            start = time.time()
            print("Sizing the options\n")
            df_chains = pd.read_pickle(fspath+'chains.pkl')
            df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')
            df_sized=sized_nse(ib, df_chains, df_ohlcs)
            print(f"\nOptions sized in {codetime(time.time()-start)}\n")
            
        elif userip == 4: # Target prepration
            start = time.time()
            print("Preparing targets\n")
            df_chains = pd.read_pickle(fspath+'chains.pkl')
            df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')
            df_sized = pd.read_pickle(fspath+'sized_nse.pkl')
            df_targets = target_nse(ib, df_sized, blacklist, nse_assignment_limit)
            print(f"\nMade SELL targets in {codetime(time.time()-start)}\n")
            
        elif userip == 5: # Place morning trades
            start = time.time()
            print("Trading in the morning\n")
            df_chains = pd.read_pickle(fspath+'chains.pkl')
            df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')
            df_sized = pd.read_pickle(fspath+'sized_nse.pkl')            
            df_targets = pd.read_pickle(fspath+'targets.pkl')
            df_buy = workout_nse(ib)
            
            sell_tb = sells(ib, df_targets, exchange)
            buy_tb = buys(ib, df_buy, exchange)
            
            # cancel all existing trades
            ib.reqGlobalCancel()            
            
            # place the morning trades            
            morning_trades = place_morning_trades(ib, sell_tb=sell_tb, buy_tb=buy_tb)
            print(f"\nCompleted morning trades in {codetime(time.time()-start)}\n")
            
        elif userip == 6: # Workout closing trades for new fills
            start = time.time()
            print("Closing new fills\n")
            
            df_buy = workout_nse(ib)
            
            if df_buy.empty:
                print("\nNothing to buy!\n")
            else:
                buy_tb = buys(ib, df_buy, exchange)
                doTrades(ib, buy_tb)
                print(f"\nFilled close BUY orders in {codetime(time.time()-start)}\n")
            
        elif userip == 7: # Capstocks BUY generation
            print("Generating BUYs for Capstocks\n")
#             cap_blacklist = 'ASHOKLEY,BHARATFORG,GRASIM,PETRONET,SUNTV,ICICIPRULI,ARVIND,BSOFT,ENGINERSIN,TATAELXSI,EICHERMOT,MOTHERSUMI,TVSMOTOR'.split(',')
            get_capstocks(cap_blacklist=cap_blacklist)
            print("Completed generating Capstocks BUY list\n")


Choose from the following numbers:
0) Prepare for morning trades
1) Chain Generation
2) OHLCs Generation
3) Size the options
4) Target preparation
5) Place morning trades
6) Workout closing trades (DYNAMIC)
7) Zip Capstocks BUYs

...Or close window to abort


6
Closing new fills



Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests



Filled close BUY orders in 0:00:17



In [16]:
desktop = r"C:/Users/kashir/Desktop/"
df1 = pd.read_pickle(desktop+'targets_fr.pkl')
df2 = pd.read_pickle(desktop+'targets_sd.pkl')
df1.merge(df2,indicator = True, how='left').loc[lambda x : x['_merge']!='both']

,symbol,undId,optId,dte,lot,right,undPrice,strike,stDev,lo52,hi52,margin,qty,remq,close,rom,sd,expPrice,_merge
7,GRASIM,250071842,376422596,30,750,P,737.30,400.0,78.670426,688.55,1141.750,100587.05,1,6,0.00,0.009072,4.287507,8.80,left_only
10,UPL,56983369,376942028,30,900,P,549.80,340.0,46.639090,366.40,698.350,91599.24,1,6,0.00,0.011954,4.498372,6.70,left_only
12,ASHOKLEY,56988221,377932761,30,6000,P,63.15,25.0,8.414715,57.90,165.550,71504.78,1,10,0.00,0.102091,4.533724,0.80,left_only
13,GRASIM,250071842,376942116,30,750,P,737.30,380.0,78.670426,688.55,1141.750,100587.05,1,6,0.00,0.009072,4.541732,8.80,left_only
24,UPL,56983369,377933098,30,900,P,549.80,330.0,46.639090,366.40,698.350,91599.24,1,6,0.00,0.011954,4.712785,6.70,left_only
25,COALINDIA,80626626,374951268,30,2200,P,188.75,125.0,13.490379,180.85,307.000,74498.50,1,9,0.00,0.035929,4.725590,2.25,left_only
28,ULTRACEMC,56983383,373127851,30,200,P,4038.40,3000.0,219.543344,3343.65,4841.050,148747.75,1,4,0.45,0.008997,4.729818,48.90,left_only
29,BRITANNIA,343576985,369072882,2,200,P,2533.85,2100.0,91.715505,2384.50,3450.075,95536.14,1,7,0.70,0.305643,4.730389,2.10,left_only
30,PFC,56986985,374953014,30,6200,P,104.75,70.0,7.341462,68.25,136.450,119767.74,1,4,0.00,0.062983,4.733390,1.25,left_only
32,ONGC,83845926,378883450,30,3750,P,123.75,77.5,9.727768,116.95,189.000,88453.23,1,8,0.00,0.051581,4.754431,1.55,left_only


In [10]:
len(df_targetfr)

359

In [14]:
len(df_targetsd)

396

In [13]:
df_targetfr.head()

,symbol,undId,optId,dte,lot,right,undPrice,strike,stDev,lo52,hi52,margin,qty,remq,close,rom,sd,expPrice
141,GRASIM,250071842,376422606,30,750,P,737.30,420.0,78.670426,688.55,1141.750,100587.05,1,6,0.00,0.009072,4.033282,8.80
34,ASIANPAIN,132214022,373161195,30,600,P,1597.55,1260.0,81.575193,1101.60,1599.450,174832.58,1,2,0.05,0.006263,4.137900,19.15
69,BRITANNIA,343576985,369072888,2,200,P,2533.85,2150.0,91.715505,2384.50,3450.075,95536.14,1,7,0.80,0.343849,4.185225,2.10
322,UJJIVAN,233076295,369067867,2,1600,P,275.15,230.0,10.699143,181.00,422.700,100677.90,1,8,0.40,1.450169,4.219964,0.50
240,ONGC,83845926,376941989,30,3750,P,123.75,82.5,9.727768,116.95,189.000,88453.23,1,8,0.00,0.051581,4.240438,1.55


In [12]:
df_targetsd.head()

,symbol,undId,optId,dte,lot,right,undPrice,strike,stDev,lo52,hi52,margin,qty,remq,close,rom,sd,expPrice
138,EQUITAS,230865250,373153355,30,4000,P,104.85,80.0,6.201193,96.60,170.45,78927.78,1,7,0.20,0.184979,4.007294,1.30
289,RAYMOND,56984932,374165698,30,800,P,574.10,400.0,43.362180,548.35,1132.40,44833.23,1,6,0.15,0.054275,4.015020,3.70
275,PEL,56986920,369066627,2,302,P,1881.40,1600.0,69.853072,1673.70,3199.75,105541.75,1,6,2.00,1.096642,4.028456,2.10
115,COLPAL,56985419,373159736,30,700,P,1201.35,1100.0,25.153961,1034.00,1330.75,154069.72,1,4,1.45,0.085681,4.029186,14.45
160,GRASIM,250071842,376422606,30,750,P,737.30,420.0,78.670426,688.55,1141.75,100587.05,1,6,0.00,0.009072,4.033282,8.80


In [15]:
len(pd.concat([df_targetfr, df_targetsd]).drop_duplicates(['optId'], keep=False))

659

In [ ]:
df_targets = pd.read_pickle(fspath+'targets.pkl')

In [ ]:
grp_opts(df_targets)

In [ ]:
with get_connected('nse', 'live') as ib:
    df_buy = workout_nse(ib)
#     sell_tb = sells(ib, df_targets, exchange)
#     sell_trades = doTrades(ib, sell_tb)

In [ ]:
df_buy